In [6]:
import numpy as np
import re

def parse_mma_dump(filename):
    """
    Parses kernel printf dump of form:

    lane  0 | A: [ a a | a a | a a | a a ]
              B: [ b b | b b ]
              C: [ c c c c ]

    Returns:
        A: (32, 4, 2)
        B: (32, 2, 2)
        C: (32, 4)
    """

    A = np.zeros((32, 4, 2), dtype=np.float32)
    B = np.zeros((32, 2, 2), dtype=np.float32)
    C = np.zeros((32, 4), dtype=np.float32)

    lane_re = re.compile(r"lane\s+(\d+)")
    float_re = re.compile(r"[-+]?\d*\.\d+|\d+")

    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # lane id
            m = lane_re.search(line)
            if not m:
                continue

            lane = int(m.group(1))

            # extract all floats in order
            vals = [float(x) for x in float_re.findall(line)]

            # Expected order:
            # A: 8 floats
            # B: 4 floats
            # C: 4 floats
            #
            # total = 16
            vals = vals[1:]
            if len(vals) != 16:
                raise ValueError(
                    f"Lane {lane}: expected 16 floats, got {len(vals)}"
                )

            # A
            A[lane, 0, 0] = vals[0]
            A[lane, 0, 1] = vals[1]

            A[lane, 1, 0] = vals[2]
            A[lane, 1, 1] = vals[3]

            A[lane, 2, 0] = vals[4]
            A[lane, 2, 1] = vals[5]

            A[lane, 3, 0] = vals[6]
            A[lane, 3, 1] = vals[7]

            # B
            B[lane, 0, 0] = vals[8]
            B[lane, 0, 1] = vals[9]

            B[lane, 1, 0] = vals[10]
            B[lane, 1, 1] = vals[11]

            # C
            C[lane, 0] = vals[12]
            C[lane, 1] = vals[13]
            C[lane, 2] = vals[14]
            C[lane, 3] = vals[15]

    return A, B, C


In [7]:
A,B,C = parse_mma_dump("data.txt")

In [11]:
print(A.shape)
print(B.shape)
print(C.shape)

(32, 4, 2)
(32, 2, 2)
(32, 4)
